In [7]:
!pip install --upgrade transformers==4.37.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 29.2 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [8]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 4.9 MB/s eta 0:00:00


In [9]:
!pip install accelerate

In [10]:
!pip install python-docx

In [2]:
from docx import Document

In [4]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [30]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained(
    "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")

In [10]:
from sentence_transformers import SentenceTransformer
encoder = SentenceTransformer('paraphrase-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.69k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
instruct_jd = '''Analyse the given job description and extract following five from it,
- Job Title
- Location
- Industry/Domain
- Education Degree Required
- Technical Skills Required
'''

In [12]:
instruct_resume = '''Analyse the given content from a resume and extract following five from it,
- Job Title
- Location
- Industry/Domain
- Education Degree
- Technical Skills
'''

In [13]:
def process_response(response):

    response_lines = response.split('\n', 4)
    processed_response = []

    for line in response_lines:

        parts = line.split(':', 1)
        if len(parts) == 2:

            value = parts[1].strip()
            value = ' '.join(value.split())
            value = value.lower()
            value = value.replace('\n', '')
            processed_response.append(value)

    return processed_response

In [24]:
def job_description_parser(jd):

  txt = ''
  with open(jd, 'r') as file:
    txt = file.read()
  print(txt)
  messages = [
    {"role": "system", "content": inst},
    {"role": "user", "content": txt}]
  text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True)

  device = "cpu"

  model_inputs = tokenizer([text], return_tensors="pt").to(device)
  generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=512, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)

  generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

  response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
  response = process_response(response)
  print(response)
  return response

In [19]:
def resume_parser(resume):

  doc = Document(resume)
  txt = ''
  for paragraph in doc.paragraphs:
    txt += paragraph.text + '\n'

  messages = [
    {"role": "system", "content": instruct_resume},
    {"role": "user", "content": txt}]
  text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True)

  device = "cpu"

  model_inputs = tokenizer([text], return_tensors="pt").to(device)
  generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=512, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)

  generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

  response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
  response = process_response(response)
  print(response)
  return response

In [16]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def job_resume_fit(jd,resume):
  parsed_jd = job_description_parser(jd)
  parsed_resume = resume_parser(resume)

  jd_embeddings = encoder.encode(parsed_jd)
  jd_embeddings = jd_embeddings / np.linalg.norm(jd_embeddings, axis=1, keepdims=True)
  print(jd_embeddings.shape)

  resume_embeddings = encoder.encode(parsed_resume)
  resume_embeddings = resume_embeddings / np.linalg.norm(resume_embeddings, axis=1, keepdims=True)
  print(resume_embeddings.shape)

  scores = []
  for i in range(5):
    jd_embedding_2d = jd_embeddings[i].reshape(1, -1)
    resume_embedding_2d = resume_embeddings[i].reshape(1, -1)
    attribute_sim = (cosine_similarity(jd_embedding_2d, resume_embedding_2d))
    scores.append(attribute_sim)
    print(attribute_sim)

  overall_score = scores[0]*0.2 + scores[1]*0.15 + scores[2]*0.2 + scores[3]*0.15 + scores[4]*0.3

  return overall_score*100

In [17]:
jd = '/content/drive/MyDrive/job_descriptions/Frontend_Developer.txt'
resume = '/content/drive/MyDrive/resumes/SDE_Resume3.docx'

In [ ]:
print(job_resume_fit(jd,resume))